[문제] Managing Conversation History
- trim_meassage()
- RunnablePassthrough
- itmegetter()

세션ID 설정하여, 대화 여러번 주고 받아야 함.(multi-turn conversation) 추가로 대화를 주고 받은 내역 출력.
LLM 모델이 과거 대화를 알지 못하는 상황을 만들기

In [1]:
## 1. 모듈 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

## 2. 환경변수
load_dotenv()

True

In [2]:
## 3. 파일 읽기
file_name ='남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law_file = file.read()

In [3]:
## 4. prompt 생성

template = [
    (
        'system',
          '''
        당신은 육아휴직 법률 전문가입니다.
        아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
        답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
        - 문서: {law}
        '''),
        ('placeholder', '{chat_history}'),
        ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

In [4]:
## 5. 인스턴스 생성

llm = ChatOpenAI(
    # model='gpt-4.1-nano',
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm

ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x000001DFCA1A4940>], client=<openai.resources.chat.completions.completions.Completions object at 0x000001DFCCB46350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001DFCCB6D0F0>, root_client=<openai.OpenAI object at 0x000001DFCCA1FC40>, root_async_client=<openai.AsyncOpenAI object at 0x000001DFCCB46410>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True)

In [5]:
## 6. trim 설정
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65, ## 최대 토큰 수 제한
    strategy='last', ## 최근 메시지부터 시작해서 자르기
    token_counter=llm, ## 모델 기준으로 토큰 수 계산
    include_system=True, ## 시스템 메시지 포함
    allow_partial=False, ## 메시지 자르기 제한
    start_on="human", ## human 메시지부터 자르기 시작
)



In [6]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
## 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history')| trimmer) 
    | prompt 
    | llm 
)

In [7]:
## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

In [8]:
## 8.함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [9]:
## 10. chain에 대화 이력 기능 래핑해서 추가

with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

In [10]:

while True:
    query = input ('육아휴직과 관련된 질문하세요 >>')

    if query.upper() == 'S':
        break
    with_message_history.invoke(
        {'law': law_file, 'query':query},
        config={'configurable': {'session_id': 'abc'}}
    )

print('\n'+'='*50+'\n')

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-I7YZBZAi6cSYieKqbDYr7iZd on tokens per min (TPM): Limit 30000, Used 13964, Requested 19039. Please try again in 6.006s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 육아휴직에 관련된 법률에 대해 궁금한 점이 있으신가요? 질문을 해주시면 성심껏 답변 드리겠습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44'}, id='run-65d57096-7784-4db8-b485-597ea1aa6178-0'), HumanMessage(content='육아휴직 조건은?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 조건에 대해 말씀드리겠습니다. 사업주는 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 육아휴직을 신청하는 경우에 이를 허용해야 합니다. 육아휴직의 기간은 1년 이내로 제한됩니다. 다만, 다음의 경우에 해당하는 근로자는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.\n\n1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모.\n2. 「한부모가족지원법」에 따른 부 또는 모.\n3. 고용노동부령으로 정하는 장애아동의 부 또는 모.\n\n또한, 사업주는 육아휴직을 이유로 근로자를 해고하거나 불리한 처우를 하면 안 됩니다. 육아휴직 기간이 끝난 후에는 근로자를 휴직 전과 동일한 업무나 같은 수준의 임금을 지급하는 직무에 복귀시켜야 하며, 휴직 기간은 근속기간에 포함시켜야 합니다. \n(「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」 제19조)', additional_kwargs={}, response_metadata={'fi

In [ ]:
store.keys()

dict_keys(['abc'])

In [ ]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]: {message.content}\n')

[HUMAN]: 

[AI]: 안녕하세요! 육아휴직에 관련된 법률에 대해 궁금한 점이 있으신가요? 질문을 해주시면 성심껏 답변 드리겠습니다.

[HUMAN]: 육아휴직 조건은?

[AI]: 육아휴직의 조건에 대해 말씀드리겠습니다. 사업주는 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 육아휴직을 신청하는 경우에 이를 허용해야 합니다. 육아휴직의 기간은 1년 이내로 제한됩니다. 다만, 다음의 경우에 해당하는 근로자는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.

1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모.
2. 「한부모가족지원법」에 따른 부 또는 모.
3. 고용노동부령으로 정하는 장애아동의 부 또는 모.

또한, 사업주는 육아휴직을 이유로 근로자를 해고하거나 불리한 처우를 하면 안 됩니다. 육아휴직 기간이 끝난 후에는 근로자를 휴직 전과 동일한 업무나 같은 수준의 임금을 지급하는 직무에 복귀시켜야 하며, 휴직 기간은 근속기간에 포함시켜야 합니다. 
(「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」 제19조)

[HUMAN]: 육아휴직 조건은?

[AI]: 육아휴직 신청 조건에 대해 설명드리겠습니다. 

육아휴직은 임신 중인 여성 근로자나 만 8세 이하 또는 초등학교 2학년 이하의 자녀(입양한 자녀 포함)를 양육하기 위해 신청할 수 있습니다. 사업주는 이러한 휴직 신청을 허용해야 합니다. 다만, 대통령령으로 정하는 경우에는 제외될 수 있습니다. 육아휴직의 기간은 최대 1년입니다. 추가로, 특정 조건을 충족하는 경우 육아휴직을 6개월 이내에서 더 사용할 수도 있습니다 (제19조 및 제19조의4제1항). 

육아휴직 사용 후에는 근로자가 동일한 업무나 같은 수준의 임금을 받는 직무로 복귀할 수 있도록 보장합니다. 이 기간은 근속기간에 포함됩니다 (제19조).

참고로, 육아휴직의 신청방법 및 절차에 대한 세부 사항은 대통령령으로 정하고 있습니다 (제19조).

[HUMAN]: